# MLforEng – Module 01: End-to-End Basics

In this module we will:

1. Train a model using the `mlforeng` package
2. Save the trained model as an artifact
3. Load the saved model and run predictions in Python
4. (Optional) Call the HTTP serving API for the same model

This module is independent — you don't need to run any other notebooks first.

In [1]:
import os
import sys
from pathlib import Path

# Assume this notebook is in MLforEng/workshops/01_end_to_end_basics
project_root = Path.cwd().parents[1]   # go up from workshops/01_end_to_end_basics -> MLforEng

print("Project root:", project_root)

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

os.chdir(project_root)
print("CWD:", os.getcwd())

from mlforeng.train import TrainConfig, train
from mlforeng.predict import load_trained_model, predict_array
from mlforeng.data import load_example_dataset


Project root: /Users/vgrover/Downloads/software/AIWorkshops/MLforEng
CWD: /Users/vgrover/Downloads/software/AIWorkshops/MLforEng


In [2]:
# Configure Training
config = TrainConfig(
    model_name="logreg",          # ALgorithm
    n_samples=500,
    n_features=10,
    test_size=0.2,
    save_model_name="module01_logreg",  # Model Name is module01_logreg placed under artifacts/pretrained
)
config


TrainConfig(dataset='synthetic', model_name='logreg', n_samples=500, n_features=10, test_size=0.2, save_model_name='module01_logreg')

In [3]:
# Train the model. Generate data → Split → Train → Evaluate → Save
results = train(config)
results


{'model_path': '/Users/vgrover/Downloads/software/AIWorkshops/MLforEng/artifacts/pretrained/module01_logreg/model.joblib',
 'config': {'dataset': 'synthetic',
  'model_name': 'logreg',
  'n_samples': 500,
  'n_features': 10,
  'test_size': 0.2,
  'save_model_name': 'module01_logreg'},
 'metrics': {'accuracy': 0.86},
 'extra': {'dataset': 'synthetic', 'n_samples': 500, 'n_features': 10}}

In [4]:
# Verify artifacts
import os

print(os.listdir("artifacts/pretrained"))
print(os.listdir("artifacts/pretrained/module01_logreg"))


['cli_logreg_test', '.DS_Store', 'commscom_rf_baseline', 'cli_rf_test', 'default_model', 'nb_logreg_test', 'commscom_rf_tuned', 'commscom_logreg_baseline', 'module01_logreg', 'quickstart_model']
['model.joblib', 'meta.json']


In [5]:
# Load persisted model. Deserializes model from disk back into memory and Access metadata (config, metrics)
loaded = load_trained_model("module01_logreg")

print("Loaded from:", loaded.path)
print("Meta keys:", loaded.meta.keys())
loaded.meta


Loaded from: /Users/vgrover/Downloads/software/AIWorkshops/MLforEng/artifacts/pretrained/module01_logreg
Meta keys: dict_keys(['config', 'metrics', 'extra'])


{'config': {'dataset': 'synthetic',
  'model_name': 'logreg',
  'n_samples': 500,
  'n_features': 10,
  'test_size': 0.2,
  'save_model_name': 'module01_logreg'},
 'metrics': {'accuracy': 0.86},
 'extra': {'dataset': 'synthetic', 'n_samples': 500, 'n_features': 10}}

In [6]:
# Figure out how many features the model expects
cfg = loaded.meta.get("config", {})
n_features = cfg.get("n_features", 10)

# Generate a fresh synthetic test set with the same feature count
splits = load_example_dataset(
    n_samples=200,
    n_features=n_features,
    test_size=0.2,
)

X_test = splits.X_test[:5]
X_test.shape


(5, 10)

In [7]:
# Run Batch prediction
preds = predict_array(loaded, X_test)
preds


array([1, 1, 1, 1, 1])

In [10]:
#Optional Step : Needs API server
# Pre-requisite is to run $python -m mlforeng.serve in your terminal 
# or $ uvicorn mlforeng.serve:app --host 0.0.0.0 --port 8000

import requests
import numpy as np

# Take a few samples from our test set and send them to the API
instances = X_test[:3].tolist()

payload = {"instances": instances}
resp = requests.post("http://127.0.0.1:8000/predict", json=payload)
resp.status_code, resp.json()


(200,
 {'model_name': 'cli_logreg_test',
  'dataset': None,
  'n_instances': 3,
  'predictions': [1, 1, 1]})